<a href="https://colab.research.google.com/github/Carlos-Pessin/Death-prediction-with-AI-model/blob/main/Projeto_Final_Aprendizagem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Bibs/')
import main

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.decomposition import FastICA
from sklearn.metrics import roc_auc_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import csv
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import tensorflow
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
#%% IMPORT DOS DADOS
X = pd.read_excel('/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/training_v3.xlsx', sheet_name='X')
y = pd.read_excel('/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/training_v3.xlsx', sheet_name='y')
X = X.to_numpy()
y = y.to_numpy()
X = np.transpose(X)
y = y[:,0]
(L,N) = np.shape(X)

In [ ]:
# REMOVE OUTLIERS GERAL
N_excl = 0
X_o = X
y_o = y
for i in range(L):
    outliers = np.array(main.t3_remoutliers(X_o[i,:], 15))
    N_excl = N_excl + len(outliers)
    if np.size(outliers)>0:
        X_o = np.delete(X_o,outliers,axis=1)
        y_o = np.delete(y_o,outliers)


# SEPARA CLASSES
y1_o = y_o>0
X1_o = X_o[:,y1_o]
X0_o = np.delete(X_o,y1_o,axis=1)
y0_o = np.delete(y_o,y1_o)
y1_o = y_o[y1_o]


# SEPARA TREINO E TESTE
X0_train, X0_test, y0_train, y0_test = train_test_split(np.transpose(X0_o), y0_o, test_size=0.1, random_state=0)
X1_train, X1_test, y1_train, y1_test = train_test_split(np.transpose(X1_o), y1_o, test_size=0.1, random_state=0)
X_train = np.transpose(np.vstack((X1_train,X0_train)))
y_train = np.concatenate((y1_train,y0_train))
X_test = np.transpose(np.vstack((X1_test,X0_test)))
y_test = np.concatenate((y1_test,y0_test))


# NORMALIZACAO
X_train_n = (X_train - np.mean(X_train))/np.std(X_train,ddof=1)
X_test_n = (X_test - np.mean(X_train))/np.std(X_train,ddof=1)

# SEPARA CLASSES
y1_train_n = y_train>0
X1_train_n = X_train_n[:,y1_train_n]
X0_train_n = np.delete(X_train_n,y1_train_n,axis=1)
y0_train_n = np.delete(y_train,y1_train_n)
y1_train_n = y_train[y1_train_n]

X_train_n=np.transpose(X_train_n)

In [ ]:
# inicializa metricas medias
SVM_sig_trainmed = {}
SVM_sig_valmed = {}

SVM_sig_trainmed['acc_train'] = 0
SVM_sig_trainmed['confMatrix_train'] = np.zeros((2,2))
SVM_sig_trainmed['TP_train'] = 0
SVM_sig_trainmed['FN_train'] = 0
SVM_sig_trainmed['TN_train'] = 0
SVM_sig_trainmed['FP_train'] = 0
SVM_sig_trainmed['Sensibilidade_train'] = 0
SVM_sig_trainmed['Especificidade_train'] = 0
SVM_sig_trainmed['Precision_train'] = 0
SVM_sig_trainmed['auc_train'] = 0
SVM_sig_valmed['acc_val'] = 0
SVM_sig_valmed['confMatrix_val'] = np.zeros((2,2))
SVM_sig_valmed['TP_val'] = 0
SVM_sig_valmed['FN_val'] = 0
SVM_sig_valmed['TN_val'] = 0
SVM_sig_valmed['FP_val'] = 0
SVM_sig_valmed['Sensibilidade_val'] = 0
SVM_sig_valmed['Especificidade_val'] = 0
SVM_sig_valmed['Precision_val'] = 0
SVM_sig_valmed['auc_val'] = 0


SVM_poly_trainmed = SVM_sig_trainmed
SVM_rbf_trainmed = SVM_sig_trainmed
RT_trainmed = SVM_sig_trainmed

SVM_poly_valmed = SVM_sig_valmed
SVM_rbf_valmed = SVM_sig_valmed
RT_valmed = SVM_sig_valmed



In [ ]:
###############################################################################################################################################
############################################-------VALIDACAO CRUZADA-------####################################################################
###############################################################################################################################################

#%% SEPARA TREINO E TESTE
n_k = 10
X_train_nk = X_train_n
skf = StratifiedKFold(n_splits=n_k, shuffle=True, random_state=0)
skf.get_n_splits(X_train_nk, y_train)
l = 0
for k, (train_index, val_index) in enumerate(skf.split(X_train_nk, y_train)):
  l = l + 1
  X_train_n, X_val_n = X_train_nk[train_index], X_train_nk[val_index]
  y_trainT, y_val_n = y_train[train_index], y_train[val_index]
  X_train_n = np.transpose(X_train_n)
  X_val_n = np.transpose(X_val_n)

    # SEPARA CLASSES
  y1_train_n = y_trainT>0
  X1_train_n = np.transpose(X_train_n[:,y1_train_n])
  X0_train_n = np.transpose(np.delete(X_train_n,y1_train_n,axis=1))
  y0_train_n = np.delete(y_trainT,y1_train_n)
  y1_train_n = y_trainT[y1_train_n]

  # REMOVE OUTLIERS TREINO
  nstd_out = 10
  N_excl_tr = 0
  for i in range(L):
      outliers0_train = np.array(main.t3_remoutliers(X0_train_n[:,i], nstd_out))
      outliers1_train = np.array(main.t3_remoutliers(X1_train_n[:,i], nstd_out))
      if np.size(outliers0_train)>0:
          X0_train_n = np.delete(X0_train_n,outliers0_train,axis=0)
          y0_train_n = np.delete(y0_train_n,outliers0_train)
      if np.size(outliers1_train)>0:
          X1_train_n = np.delete(X1_train_n,outliers1_train,axis=0)
          y1_train_n = np.delete(y1_train_n,outliers1_train)
      N_excl_tr = N_excl_tr + len(outliers0_train) + len(outliers1_train)


  # SUBAMOSTRAGEM CLASSE 0
  X0_train_n, X0_c, y0_train_n, y0_c = train_test_split(X0_train_n, y0_train_n, test_size=0.9, random_state=0)


  # DADOS DE TREINO TOTAL
  X_train_n = np.transpose(np.vstack((X0_train_n, X1_train_n)))
  y_train_n = np.concatenate((y0_train_n, y1_train_n))
  (L,N_train_n) = np.shape(X_train_n)
  (L,N_val_n) = np.shape(X_val_n)


  # TRANSFORMA ESPACO
  Ls_PCA = 55   # mse<5
  w,mse,X_train_t,v = main.t4_pca(X_train_n, Ls_PCA)
  X_val_t = np.dot(v.T,X_val_n)     # utiliza criterio dos dados de treino

  ica = FastICA(n_components=Ls_PCA)
  X_train_t = ica.fit_transform(np.transpose(X_train_t))  # estimated independent sources
  X_val_t = ica.transform(np.transpose(X_val_t))
  X_train_t = np.transpose(X_train_t)
  X_val_t = np.transpose(X_val_t)


  # SELECAO ESCALAR COM RANDOM FOREST
  arvores = RandomForestClassifier(n_estimators=200, min_samples_split=50, min_samples_leaf=20, random_state=0)
  arvores.fit(np.transpose(X_train_t), y_train_n)
  ordem = arvores.feature_importances_
  ordem = np.argsort(-ordem)
  Ls_SE = 40    # numero de caracteristicas a selecionar
  X_train_s = np.zeros((Ls_SE,N_train_n))
  X_val_s = np.zeros((Ls_SE,N_val_n))
  for i in range(Ls_SE):
      X_train_s[i,:] = X_train_t[ordem[i],:]
      X_val_s[i,:] = X_val_t[ordem[i],:]    # utiliza criterio dos dados de treino


  # SEPARA CLASSES
  y1_train_t = y_train_n>0
  X1_train_t = X_train_s[:,y1_train_t]
  X0_train_t = np.delete(X_train_s,y1_train_t,axis=1)
  y0_train_t = np.delete(y_train_n,y1_train_t)
  y1_train_t = y_train_n[y1_train_t]
  y1_val_t = y_val_n>0
  X1_val_t = X_val_t[:,y1_val_t]
  X0_val_t = np.delete(X_val_t,y1_val_t,axis=1)
  y0_val_t = np.delete(y_val_n,y1_val_t)
  y1_val_t = y_val_n[y1_val_t]


  #%% RANDOM FOREST
  n_estimators = 1000
  min_samples_split = 50
  min_samples_leaf = 5
  arvores = RandomForestClassifier(n_estimators=n_estimators, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, random_state=0)
  arvores.fit(np.transpose(X_train_s), y_train_n)
  y_train_pred = arvores.predict(np.transpose(X_train_s))
  y_val_pred = arvores.predict(np.transpose(X_val_s))


  # AVALIACAO
  param={}
  param['outlier'] = nstd_out
  param['PCA-Ls'] = Ls_PCA
  param['RT_Selec'] = Ls_SE
  param['classificador'] = 'Random Forest'
  param['n_estimators'] = n_estimators
  param['min_samples_split'] = min_samples_split
  param['min_samples_leaf'] = min_samples_leaf
  train_aval={}
  val_aval={}
  train_aval['acc_train'] = accuracy_score(y_train_n, y_train_pred)
  val_aval['acc_val'] = accuracy_score(y_val_n, y_val_pred)
  train_aval['confMatrix_train'] = confusion_matrix(y_train_n, y_train_pred)
  val_aval['confMatrix_val'] = confusion_matrix(y_val_n, y_val_pred)
  c_target=1
  train_aval['TP_train'] = train_aval['confMatrix_train'][c_target,c_target]
  train_aval['FN_train'] = np.sum(train_aval['confMatrix_train'][c_target,:]) - train_aval['TP_train']
  train_aval['TN_train'] = np.sum(train_aval['confMatrix_train'] * np.identity(2)) - train_aval['TP_train']
  train_aval['FP_train'] = np.sum(train_aval['confMatrix_train']) - train_aval['TP_train'] - train_aval['FN_train'] - train_aval['TN_train']
  train_aval['Sensibilidade_train'] = train_aval['TP_train']/(train_aval['TP_train']+train_aval['FN_train'])
  train_aval['Especificidade_train'] = train_aval['TN_train']/(train_aval['TN_train']+train_aval['FP_train'])
  train_aval['Precision_train'] = train_aval['TP_train']/(train_aval['TP_train']+train_aval['FP_train'])
  train_aval['auc_train'] = roc_auc_score(y_train_n, y_train_pred)
  val_aval['TP_val'] = val_aval['confMatrix_val'][c_target,c_target]
  val_aval['FN_val'] = np.sum(val_aval['confMatrix_val'][c_target,:]) - val_aval['TP_val']
  val_aval['TN_val'] = np.sum(val_aval['confMatrix_val'] * np.identity(2)) - val_aval['TP_val']
  val_aval['FP_val'] = np.sum(val_aval['confMatrix_val']) - val_aval['TP_val'] - val_aval['FN_val'] - val_aval['TN_val']
  val_aval['Sensibilidade_val'] = val_aval['TP_val']/(val_aval['TP_val']+val_aval['FN_val'])
  val_aval['Especificidade_val'] = val_aval['TN_val']/(val_aval['TN_val']+val_aval['FP_val'])
  val_aval['Precision_val'] = val_aval['TP_val']/(val_aval['TP_val']+val_aval['FP_val'])
  val_aval['auc_val'] = roc_auc_score(y_val_n, y_val_pred)
  RT_trainmed['acc_train'] = RT_trainmed['acc_train'] + train_aval['acc_train']/n_k
  RT_trainmed['confMatrix_train'] = RT_trainmed['confMatrix_train'] + train_aval['confMatrix_train']/n_k
  RT_trainmed['TP_train'] = RT_trainmed['TP_train'] + train_aval['TP_train']/n_k
  RT_trainmed['FN_train'] = RT_trainmed['FN_train'] + train_aval['FN_train']/n_k
  RT_trainmed['TN_train'] = RT_trainmed['TN_train'] + train_aval['TN_train']/n_k
  RT_trainmed['FP_train'] = RT_trainmed['FP_train'] + train_aval['FP_train']/n_k
  RT_trainmed['Sensibilidade_train'] = RT_trainmed['Sensibilidade_train'] + train_aval['Sensibilidade_train']/n_k
  RT_trainmed['Especificidade_train'] = RT_trainmed['Especificidade_train'] + train_aval['Especificidade_train']/n_k
  RT_trainmed['Precision_train'] = RT_trainmed['Precision_train'] + train_aval['Precision_train']/n_k
  RT_trainmed['auc_train'] = RT_trainmed['auc_train'] + train_aval['auc_train']/n_k
  RT_valmed['acc_val'] = RT_valmed['acc_val'] + val_aval['acc_val']/n_k
  RT_valmed['confMatrix_val'] = RT_valmed['confMatrix_val'] + val_aval['confMatrix_val']/n_k
  RT_valmed['TP_val'] = RT_valmed['TP_val'] + val_aval['TP_val']/n_k
  RT_valmed['FN_val'] = RT_valmed['FN_val'] + val_aval['FN_val']/n_k
  RT_valmed['TN_val'] = RT_valmed['TN_val'] + val_aval['TN_val']/n_k
  RT_valmed['FP_val'] = RT_valmed['FP_val'] + val_aval['FP_val']/n_k
  RT_valmed['Sensibilidade_val'] = RT_valmed['Sensibilidade_val'] + val_aval['Sensibilidade_val']/n_k
  RT_valmed['Especificidade_val'] = RT_valmed['Especificidade_val'] + val_aval['Especificidade_val']/n_k
  RT_valmed['Precision_val'] = RT_valmed['Precision_val'] + val_aval['Precision_val']/n_k
  RT_valmed['auc_val'] = RT_valmed['auc_val'] + val_aval['auc_val']/n_k



  # EXPORTA RESULTADOS
  filename = '/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/aval/' + param['classificador']  + str(l) + '_train_aval.csv'
  with open(filename, 'w', newline='') as csv_file:
      writer = csv.writer(csv_file)
      for key, value in train_aval.items():
          writer.writerow([key, value])

  filename = '/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/aval/' + param['classificador']  + str(l) + '_val_aval.csv'
  with open(filename, 'w', newline='') as csv_file:
      writer = csv.writer(csv_file)
      for key, value in val_aval.items():
          writer.writerow([key, value])

  filename = '/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/aval/' + param['classificador']  + str(l) + '_parametros.csv'
  with open(filename, 'w', newline='') as csv_file:
      writer = csv.writer(csv_file)
      for key, value in param.items():
          writer.writerow([key, value])


  # SVM RBF
  C = 30
  kernel = 'rbf'
  tol = 0.01
  gamma = 0.5
  clf = SVC(C=C, kernel=kernel, tol=tol, gamma=gamma)
  clf.fit(np.transpose(X_train_s), y_train_n)
  y_train_pred = clf.predict(np.transpose(X_train_s))
  y_val_pred = clf.predict(np.transpose(X_val_s))


  # AVALIACAO
  param={}
  param['outlier'] = nstd_out
  param['PCA-Ls'] = Ls_PCA
  param['RT_Selec'] = Ls_SE
  param['classificador'] = 'SVM - rbf'
  param['C'] = C
  param['kernel'] = kernel
  param['tol'] = tol
  param['gamma'] = gamma
  train_aval={}
  val_aval={}
  train_aval['acc_train'] = accuracy_score(y_train_n, y_train_pred)
  val_aval['acc_val'] = accuracy_score(y_val_n, y_val_pred)
  train_aval['confMatrix_train'] = confusion_matrix(y_train_n, y_train_pred)
  val_aval['confMatrix_val'] = confusion_matrix(y_val_n, y_val_pred)
  c_target=1
  train_aval['TP_train'] = train_aval['confMatrix_train'][c_target,c_target]
  train_aval['FN_train'] = np.sum(train_aval['confMatrix_train'][c_target,:]) - train_aval['TP_train']
  train_aval['TN_train'] = np.sum(train_aval['confMatrix_train'] * np.identity(2)) - train_aval['TP_train']
  train_aval['FP_train'] = np.sum(train_aval['confMatrix_train']) - train_aval['TP_train'] - train_aval['FN_train'] - train_aval['TN_train']
  train_aval['Sensibilidade_train'] = train_aval['TP_train']/(train_aval['TP_train']+train_aval['FN_train'])
  train_aval['Especificidade_train'] = train_aval['TN_train']/(train_aval['TN_train']+train_aval['FP_train'])
  train_aval['Precision_train'] = train_aval['TP_train']/(train_aval['TP_train']+train_aval['FP_train'])
  train_aval['auc_train'] = roc_auc_score(y_train_n, y_train_pred)
  val_aval['TP_val'] = val_aval['confMatrix_val'][c_target,c_target]
  val_aval['FN_val'] = np.sum(val_aval['confMatrix_val'][c_target,:]) - val_aval['TP_val']
  val_aval['TN_val'] = np.sum(val_aval['confMatrix_val'] * np.identity(2)) - val_aval['TP_val']
  val_aval['FP_val'] = np.sum(val_aval['confMatrix_val']) - val_aval['TP_val'] - val_aval['FN_val'] - val_aval['TN_val']
  val_aval['Sensibilidade_val'] = val_aval['TP_val']/(val_aval['TP_val']+val_aval['FN_val'])
  val_aval['Especificidade_val'] = val_aval['TN_val']/(val_aval['TN_val']+val_aval['FP_val'])
  val_aval['Precision_val'] = val_aval['TP_val']/(val_aval['TP_val']+val_aval['FP_val'])
  val_aval['auc_val'] = roc_auc_score(y_val_n, y_val_pred)
  SVM_rbf_trainmed['acc_train'] = SVM_rbf_trainmed['acc_train'] + train_aval['acc_train']/n_k
  SVM_rbf_trainmed['confMatrix_train'] = SVM_rbf_trainmed['confMatrix_train'] + train_aval['confMatrix_train']/n_k
  SVM_rbf_trainmed['TP_train'] = SVM_rbf_trainmed['TP_train'] + train_aval['TP_train']/n_k
  SVM_rbf_trainmed['FN_train'] = SVM_rbf_trainmed['FN_train'] + train_aval['FN_train']/n_k
  SVM_rbf_trainmed['TN_train'] = SVM_rbf_trainmed['TN_train'] + train_aval['TN_train']/n_k
  SVM_rbf_trainmed['FP_train'] = SVM_rbf_trainmed['FP_train'] + train_aval['FP_train']/n_k
  SVM_rbf_trainmed['Sensibilidade_train'] = SVM_rbf_trainmed['Sensibilidade_train'] + train_aval['Sensibilidade_train']/n_k
  SVM_rbf_trainmed['Especificidade_train'] = SVM_rbf_trainmed['Especificidade_train'] + train_aval['Especificidade_train']/n_k
  SVM_rbf_trainmed['Precision_train'] = SVM_rbf_trainmed['Precision_train'] + train_aval['Precision_train']/n_k
  SVM_rbf_trainmed['auc_train'] = SVM_rbf_trainmed['auc_train'] + train_aval['auc_train']/n_k
  SVM_rbf_valmed['acc_val'] = SVM_rbf_valmed['acc_val'] + val_aval['acc_val']/n_k
  SVM_rbf_valmed['confMatrix_val'] = SVM_rbf_valmed['confMatrix_val'] + val_aval['confMatrix_val']/n_k
  SVM_rbf_valmed['TP_val'] = SVM_rbf_valmed['TP_val'] + val_aval['TP_val']/n_k
  SVM_rbf_valmed['FN_val'] = SVM_rbf_valmed['FN_val'] + val_aval['FN_val']/n_k
  SVM_rbf_valmed['TN_val'] = SVM_rbf_valmed['TN_val'] + val_aval['TN_val']/n_k
  SVM_rbf_valmed['FP_val'] = SVM_rbf_valmed['FP_val'] + val_aval['FP_val']/n_k
  SVM_rbf_valmed['Sensibilidade_val'] = SVM_rbf_valmed['Sensibilidade_val'] + val_aval['Sensibilidade_val']/n_k
  SVM_rbf_valmed['Especificidade_val'] = SVM_rbf_valmed['Especificidade_val'] + val_aval['Especificidade_val']/n_k
  SVM_rbf_valmed['Precision_val'] = SVM_rbf_valmed['Precision_val'] + val_aval['Precision_val']/n_k
  SVM_rbf_valmed['auc_val'] = SVM_rbf_valmed['auc_val'] + val_aval['auc_val']/n_k


  # EXPORTA RESULTADOS
  filename = '/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/aval/' + param['classificador']  + str(l) + '_train_aval.csv'
  with open(filename, 'w', newline='') as csv_file:
      writer = csv.writer(csv_file)
      for key, value in train_aval.items():
          writer.writerow([key, value])

  filename = '/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/aval/' + param['classificador']  + str(l) + '_val_aval.csv'
  with open(filename, 'w', newline='') as csv_file:
      writer = csv.writer(csv_file)
      for key, value in val_aval.items():
          writer.writerow([key, value])

  filename = '/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/aval/' + param['classificador']  + str(l) + '_parametros.csv'
  with open(filename, 'w', newline='') as csv_file:
      writer = csv.writer(csv_file)
      for key, value in param.items():
          writer.writerow([key, value])


  # SVM POL
  C = 20
  kernel = 'poly'
  degree = 4    #melhor no treino, pior no teste, sobreajuste
  coef0 = 2
  clf = SVC(C=C, kernel=kernel, degree=degree, coef0=coef0)
  clf.fit(np.transpose(X_train_t), y_train_n)
  y_train_pred = clf.predict(np.transpose(X_train_t))
  y_val_pred = clf.predict(np.transpose(X_val_t))


  # AVALIACAO
  param={}
  param['outlier'] = nstd_out
  param['PCA-Ls'] = Ls_PCA
  param['RT_Selec'] = Ls_SE
  param['classificador'] = 'SVM - ploy'
  param['C'] = C
  param['kernel'] = kernel
  param['degree'] = degree
  param['coef0'] = coef0
  train_aval={}
  val_aval={}
  train_aval['acc_train'] = accuracy_score(y_train_n, y_train_pred)
  val_aval['acc_val'] = accuracy_score(y_val_n, y_val_pred)
  train_aval['confMatrix_train'] = confusion_matrix(y_train_n, y_train_pred)
  val_aval['confMatrix_val'] = confusion_matrix(y_val_n, y_val_pred)
  c_target=1
  train_aval['TP_train'] = train_aval['confMatrix_train'][c_target,c_target]
  train_aval['FN_train'] = np.sum(train_aval['confMatrix_train'][c_target,:]) - train_aval['TP_train']
  train_aval['TN_train'] = np.sum(train_aval['confMatrix_train'] * np.identity(2)) - train_aval['TP_train']
  train_aval['FP_train'] = np.sum(train_aval['confMatrix_train']) - train_aval['TP_train'] - train_aval['FN_train'] - train_aval['TN_train']
  train_aval['Sensibilidade_train'] = train_aval['TP_train']/(train_aval['TP_train']+train_aval['FN_train'])
  train_aval['Especificidade_train'] = train_aval['TN_train']/(train_aval['TN_train']+train_aval['FP_train'])
  train_aval['Precision_train'] = train_aval['TP_train']/(train_aval['TP_train']+train_aval['FP_train'])
  train_aval['auc_train'] = roc_auc_score(y_train_n, y_train_pred)
  val_aval['TP_val'] = val_aval['confMatrix_val'][c_target,c_target]
  val_aval['FN_val'] = np.sum(val_aval['confMatrix_val'][c_target,:]) - val_aval['TP_val']
  val_aval['TN_val'] = np.sum(val_aval['confMatrix_val'] * np.identity(2)) - val_aval['TP_val']
  val_aval['FP_val'] = np.sum(val_aval['confMatrix_val']) - val_aval['TP_val'] - val_aval['FN_val'] - val_aval['TN_val']
  val_aval['Sensibilidade_val'] = val_aval['TP_val']/(val_aval['TP_val']+val_aval['FN_val'])
  val_aval['Especificidade_val'] = val_aval['TN_val']/(val_aval['TN_val']+val_aval['FP_val'])
  val_aval['Precision_val'] = val_aval['TP_val']/(val_aval['TP_val']+val_aval['FP_val'])
  val_aval['auc_val'] = roc_auc_score(y_val_n, y_val_pred)
  SVM_poly_trainmed['acc_train'] = SVM_poly_trainmed['acc_train'] + train_aval['acc_train']/n_k
  SVM_poly_trainmed['confMatrix_train'] = SVM_poly_trainmed['confMatrix_train'] + train_aval['confMatrix_train']/n_k
  SVM_poly_trainmed['TP_train'] = SVM_poly_trainmed['TP_train'] + train_aval['TP_train']/n_k
  SVM_poly_trainmed['FN_train'] = SVM_poly_trainmed['FN_train'] + train_aval['FN_train']/n_k
  SVM_poly_trainmed['TN_train'] = SVM_poly_trainmed['TN_train'] + train_aval['TN_train']/n_k
  SVM_poly_trainmed['FP_train'] = SVM_poly_trainmed['FP_train'] + train_aval['FP_train']/n_k
  SVM_poly_trainmed['Sensibilidade_train'] = SVM_poly_trainmed['Sensibilidade_train'] + train_aval['Sensibilidade_train']/n_k
  SVM_poly_trainmed['Especificidade_train'] = SVM_poly_trainmed['Especificidade_train'] + train_aval['Especificidade_train']/n_k
  SVM_poly_trainmed['Precision_train'] = SVM_poly_trainmed['Precision_train'] + train_aval['Precision_train']/n_k
  SVM_poly_trainmed['auc_train'] = SVM_poly_trainmed['auc_train'] + train_aval['auc_train']/n_k
  SVM_poly_valmed['acc_val'] = SVM_poly_valmed['acc_val'] + val_aval['acc_val']/n_k
  SVM_poly_valmed['confMatrix_val'] = SVM_poly_valmed['confMatrix_val'] + val_aval['confMatrix_val']/n_k
  SVM_poly_valmed['TP_val'] = SVM_poly_valmed['TP_val'] + val_aval['TP_val']/n_k
  SVM_poly_valmed['FN_val'] = SVM_poly_valmed['FN_val'] + val_aval['FN_val']/n_k
  SVM_poly_valmed['TN_val'] = SVM_poly_valmed['TN_val'] + val_aval['TN_val']/n_k
  SVM_poly_valmed['FP_val'] = SVM_poly_valmed['FP_val'] + val_aval['FP_val']/n_k
  SVM_poly_valmed['Sensibilidade_val'] = SVM_poly_valmed['Sensibilidade_val'] + val_aval['Sensibilidade_val']/n_k
  SVM_poly_valmed['Especificidade_val'] = SVM_poly_valmed['Especificidade_val'] + val_aval['Especificidade_val']/n_k
  SVM_poly_valmed['Precision_val'] = SVM_poly_valmed['Precision_val'] + val_aval['Precision_val']/n_k
  SVM_poly_valmed['auc_val'] = SVM_poly_valmed['auc_val'] + val_aval['auc_val']/n_k

  # EXPORTA RESULTADOS
  filename = '/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/aval/' + param['classificador']  + str(l) + '_train_aval.csv'
  with open(filename, 'w', newline='') as csv_file:
      writer = csv.writer(csv_file)
      for key, value in train_aval.items():
          writer.writerow([key, value])

  filename = '/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/aval/' + param['classificador']  + str(l) + '_val_aval.csv'
  with open(filename, 'w', newline='') as csv_file:
      writer = csv.writer(csv_file)
      for key, value in val_aval.items():
          writer.writerow([key, value])

  filename = '/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/aval/' + param['classificador']  + str(l) + '_parametros.csv'
  with open(filename, 'w', newline='') as csv_file:
      writer = csv.writer(csv_file)
      for key, value in param.items():
          writer.writerow([key, value])


  # SVM SIGMOIDE
  C = 30
  kernel = 'sigmoid'
  coef0 = -1
  clf = SVC(C=C, kernel=kernel, coef0=coef0)
  clf.fit(np.transpose(X_train_t), y_train_n)
  y_train_pred = clf.predict(np.transpose(X_train_t))
  y_val_pred = clf.predict(np.transpose(X_val_t))


  # AVALIACAO
  param={}
  param['outlier'] = nstd_out
  param['PCA-Ls'] = Ls_PCA
  param['RT_Selec'] = Ls_SE
  param['classificador'] = 'SVM - sig'
  param['C'] = C
  param['kernel'] = kernel
  param['degree'] = degree
  param['coef0'] = coef0
  train_aval={}
  val_aval={}
  train_aval['acc_train'] = accuracy_score(y_train_n, y_train_pred)
  val_aval['acc_val'] = accuracy_score(y_val_n, y_val_pred)
  train_aval['confMatrix_train'] = confusion_matrix(y_train_n, y_train_pred)
  val_aval['confMatrix_val'] = confusion_matrix(y_val_n, y_val_pred)
  c_target=1
  train_aval['TP_train'] = train_aval['confMatrix_train'][c_target,c_target]
  train_aval['FN_train'] = np.sum(train_aval['confMatrix_train'][c_target,:]) - train_aval['TP_train']
  train_aval['TN_train'] = np.sum(train_aval['confMatrix_train'] * np.identity(2)) - train_aval['TP_train']
  train_aval['FP_train'] = np.sum(train_aval['confMatrix_train']) - train_aval['TP_train'] - train_aval['FN_train'] - train_aval['TN_train']
  train_aval['Sensibilidade_train'] = train_aval['TP_train']/(train_aval['TP_train']+train_aval['FN_train'])
  train_aval['Especificidade_train'] = train_aval['TN_train']/(train_aval['TN_train']+train_aval['FP_train'])
  train_aval['Precision_train'] = train_aval['TP_train']/(train_aval['TP_train']+train_aval['FP_train'])
  train_aval['auc_train'] = roc_auc_score(y_train_n, y_train_pred)
  val_aval['TP_val'] = val_aval['confMatrix_val'][c_target,c_target]
  val_aval['FN_val'] = np.sum(val_aval['confMatrix_val'][c_target,:]) - val_aval['TP_val']
  val_aval['TN_val'] = np.sum(val_aval['confMatrix_val'] * np.identity(2)) - val_aval['TP_val']
  val_aval['FP_val'] = np.sum(val_aval['confMatrix_val']) - val_aval['TP_val'] - val_aval['FN_val'] - val_aval['TN_val']
  val_aval['Sensibilidade_val'] = val_aval['TP_val']/(val_aval['TP_val']+val_aval['FN_val'])
  val_aval['Especificidade_val'] = val_aval['TN_val']/(val_aval['TN_val']+val_aval['FP_val'])
  val_aval['Precision_val'] = val_aval['TP_val']/(val_aval['TP_val']+val_aval['FP_val'])
  val_aval['auc_val'] = roc_auc_score(y_val_n, y_val_pred)
  SVM_sig_trainmed['acc_train'] = SVM_sig_trainmed['acc_train'] + train_aval['acc_train']/n_k
  SVM_sig_trainmed['confMatrix_train'] = SVM_sig_trainmed['confMatrix_train'] + train_aval['confMatrix_train']/n_k
  SVM_sig_trainmed['TP_train'] = SVM_sig_trainmed['TP_train'] + train_aval['TP_train']/n_k
  SVM_sig_trainmed['FN_train'] = SVM_sig_trainmed['FN_train'] + train_aval['FN_train']/n_k
  SVM_sig_trainmed['TN_train'] = SVM_sig_trainmed['TN_train'] + train_aval['TN_train']/n_k
  SVM_sig_trainmed['FP_train'] = SVM_sig_trainmed['FP_train'] + train_aval['FP_train']/n_k
  SVM_sig_trainmed['Sensibilidade_train'] = SVM_sig_trainmed['Sensibilidade_train'] + train_aval['Sensibilidade_train']/n_k
  SVM_sig_trainmed['Especificidade_train'] = SVM_sig_trainmed['Especificidade_train'] + train_aval['Especificidade_train']/n_k
  SVM_sig_trainmed['Precision_train'] = SVM_sig_trainmed['Precision_train'] + train_aval['Precision_train']/n_k
  SVM_sig_trainmed['auc_train'] = SVM_sig_trainmed['auc_train'] + train_aval['auc_train']/n_k
  SVM_sig_valmed['acc_val'] = SVM_sig_valmed['acc_val'] + val_aval['acc_val']/n_k
  SVM_sig_valmed['confMatrix_val'] = SVM_sig_valmed['confMatrix_val'] + val_aval['confMatrix_val']/n_k
  SVM_sig_valmed['TP_val'] = SVM_sig_valmed['TP_val'] + val_aval['TP_val']/n_k
  SVM_sig_valmed['FN_val'] = SVM_sig_valmed['FN_val'] + val_aval['FN_val']/n_k
  SVM_sig_valmed['TN_val'] = SVM_sig_valmed['TN_val'] + val_aval['TN_val']/n_k
  SVM_sig_valmed['FP_val'] = SVM_sig_valmed['FP_val'] + val_aval['FP_val']/n_k
  SVM_sig_valmed['Sensibilidade_val'] = SVM_sig_valmed['Sensibilidade_val'] + val_aval['Sensibilidade_val']/n_k
  SVM_sig_valmed['Especificidade_val'] = SVM_sig_valmed['Especificidade_val'] + val_aval['Especificidade_val']/n_k
  SVM_sig_valmed['Precision_val'] = SVM_sig_valmed['Precision_val'] + val_aval['Precision_val']/n_k
  SVM_sig_valmed['auc_val'] = SVM_sig_valmed['auc_val'] + val_aval['auc_val']/n_k

  # EXPORTA RESULTADOS
  filename = '/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/aval/' + param['classificador']  + str(l) + '_train_aval.csv'
  with open(filename, 'w', newline='') as csv_file:
      writer = csv.writer(csv_file)
      for key, value in train_aval.items():
          writer.writerow([key, value])

  filename = '/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/aval/' + param['classificador']  + str(l) + '_val_aval.csv'
  with open(filename, 'w', newline='') as csv_file:
      writer = csv.writer(csv_file)
      for key, value in val_aval.items():
          writer.writerow([key, value])

  filename = '/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/aval/' + param['classificador']  + str(l) + '_parametros.csv'
  with open(filename, 'w', newline='') as csv_file:
      writer = csv.writer(csv_file)
      for key, value in param.items():
          writer.writerow([key, value])



# EXPORTA RESULTADOS
filename = '/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/aval/RT_trainmed.csv'
with open(filename, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in RT_trainmed.items():
        writer.writerow([key, value])

filename = '/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/aval/SVM_poly_trainmed.csv'
with open(filename, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in SVM_poly_trainmed.items():
        writer.writerow([key, value])

filename = '/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/aval/SVM_rbf_trainmed.csv'
with open(filename, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in SVM_rbf_trainmed.items():
        writer.writerow([key, value])

filename = '/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/aval/SVM_sig_trainmed.csv'
with open(filename, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in SVM_sig_trainmed.items():
        writer.writerow([key, value])

filename = '/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/aval/RT_valmed.csv'
with open(filename, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in RT_valmed.items():
        writer.writerow([key, value])

filename = '/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/aval/SVM_poly_valmed.csv'
with open(filename, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in SVM_poly_valmed.items():
        writer.writerow([key, value])

filename = '/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/aval/SVM_rbf_valmed.csv'
with open(filename, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in SVM_rbf_valmed.items():
        writer.writerow([key, value])

filename = '/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/aval/SVM_sig_valmed.csv'
with open(filename, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in SVM_sig_valmed.items():
        writer.writerow([key, value])

In [ ]:
X_train_n = np.transpose(X_train_n)
X_test_n = np.transpose(X_test_n)
y_test_n = y_test
y_trainT = y_train
  # SEPARA CLASSES
y1_train_n = y_trainT>0
X1_train_n = np.transpose(X_train_n[:,y1_train_n])
X0_train_n = np.transpose(np.delete(X_train_n,y1_train_n,axis=1))
y0_train_n = np.delete(y_trainT,y1_train_n)
y1_train_n = y_trainT[y1_train_n]

# REMOVE OUTLIERS TREINO
nstd_out = 10
N_excl_tr = 0
for i in range(L):
    outliers0_train = np.array(main.t3_remoutliers(X0_train_n[:,i], nstd_out))
    outliers1_train = np.array(main.t3_remoutliers(X1_train_n[:,i], nstd_out))
    if np.size(outliers0_train)>0:
        X0_train_n = np.delete(X0_train_n,outliers0_train,axis=0)
        y0_train_n = np.delete(y0_train_n,outliers0_train)
    if np.size(outliers1_train)>0:
        X1_train_n = np.delete(X1_train_n,outliers1_train,axis=0)
        y1_train_n = np.delete(y1_train_n,outliers1_train)
    N_excl_tr = N_excl_tr + len(outliers0_train) + len(outliers1_train)


# SUBAMOSTRAGEM CLASSE 0
X0_train_n, X0_c, y0_train_n, y0_c = train_test_split(X0_train_n, y0_train_n, test_size=0.9, random_state=0)


# DADOS DE TREINO TOTAL
X_train_n = np.transpose(np.vstack((X0_train_n, X1_train_n)))
y_train_n = np.concatenate((y0_train_n, y1_train_n))
(L,N_train_n) = np.shape(X_train_n)
(L,N_test_n) = np.shape(X_test_n)


# TRANSFORMA ESPACO
Ls_PCA = 55   # mse<5
w,mse,X_train_t,v = main.t4_pca(X_train_n, Ls_PCA)
X_test_t = np.dot(v.T,X_test_n)     # utiliza criterio dos dados de treino

ica = FastICA(n_components=Ls_PCA)
X_train_t = ica.fit_transform(np.transpose(X_train_t))  # estimated independent sources
X_test_t = ica.transform(np.transpose(X_test_t))
X_train_t = np.transpose(X_train_t)
X_test_t = np.transpose(X_test_t)


# SELECAO ESCALAR COM RANDOM FOREST
arvores = RandomForestClassifier(n_estimators=200, min_samples_split=50, min_samples_leaf=20, random_state=0)
arvores.fit(np.transpose(X_train_t), y_train_n)
ordem = arvores.feature_importances_
ordem = np.argsort(-ordem)
Ls_SE = 40    # numero de caracteristicas a selecionar
X_train_s = np.zeros((Ls_SE,N_train_n))
X_test_s = np.zeros((Ls_SE,N_test_n))
for i in range(Ls_SE):
    X_train_s[i,:] = X_train_t[ordem[i],:]
    X_test_s[i,:] = X_test_t[ordem[i],:]    # utiliza criterio dos dados de treino


# SEPARA CLASSES
y1_train_t = y_train_n>0
X1_train_t = X_train_s[:,y1_train_t]
X0_train_t = np.delete(X_train_s,y1_train_t,axis=1)
y0_train_t = np.delete(y_train_n,y1_train_t)
y1_train_t = y_train_n[y1_train_t]
y1_test_t = y_test_n>0
X1_test_t = X_test_t[:,y1_test_t]
X0_test_t = np.delete(X_test_t,y1_test_t,axis=1)
y0_test_t = np.delete(y_test_n,y1_test_t)
y1_test_t = y_test_n[y1_test_t]


#%% RANDOM FOREST
n_estimators = 1000
min_samples_split = 50
min_samples_leaf = 5
arvores = RandomForestClassifier(n_estimators=n_estimators, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, random_state=0)
arvores.fit(np.transpose(X_train_s), y_train_n)
y_train_pred = arvores.predict(np.transpose(X_train_s))
y_test_pred = arvores.predict(np.transpose(X_test_s))


# AVALIACAO
test_atest={}
test_atest['acc_test'] = accuracy_score(y_test_n, y_test_pred)
test_atest['confMatrix_test'] = confusion_matrix(y_test_n, y_test_pred)
c_target=1
test_atest['TP_test'] = test_atest['confMatrix_test'][c_target,c_target]
test_atest['FN_test'] = np.sum(test_atest['confMatrix_test'][c_target,:]) - test_atest['TP_test']
test_atest['TN_test'] = np.sum(test_atest['confMatrix_test'] * np.identity(2)) - test_atest['TP_test']
test_atest['FP_test'] = np.sum(test_atest['confMatrix_test']) - test_atest['TP_test'] - test_atest['FN_test'] - test_atest['TN_test']
test_atest['Sensibilidade_test'] = test_atest['TP_test']/(test_atest['TP_test']+test_atest['FN_test'])
test_atest['Especificidade_test'] = test_atest['TN_test']/(test_atest['TN_test']+test_atest['FP_test'])
test_atest['Precision_test'] = test_atest['TP_test']/(test_atest['TP_test']+test_atest['FP_test'])
test_atest['auc_test'] = roc_auc_score(y_test_n, y_test_pred)

/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(


In [ ]:
filename = '/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/aval/Avaliacao Teste.csv'
with open(filename, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in test_atest.items():
        writer.writerow([key, value])

In [ ]:

# SEPARA TREINO E VALIDAÇÃO
X0_train_n, X0_val_n, y0_train_n, y0_val_n = train_test_split(np.transpose(X0_train_n), y0_train_n, test_size=0.1, random_state=0)
X1_train_n, X1_val_n, y1_train_n, y1_val_n = train_test_split(np.transpose(X1_train_n), y1_train_n, test_size=0.1, random_state=0)
X_train_n = np.transpose(np.vstack((X1_train_n,X0_train_n)))
y_train_n = np.concatenate((y1_train_n,y0_train_n))
X_val_n = np.transpose(np.vstack((X1_val_n,X0_val_n)))
y_val_n = np.concatenate((y1_val_n,y0_val_n))


# REMOVE OUTLIERS TREINO
nstd_out = 10
N_excl_tr = 0
for i in range(L):
    outliers0_train = np.array(main.t3_remoutliers(X0_train_n[:,i], nstd_out))
    outliers1_train = np.array(main.t3_remoutliers(X1_train_n[:,i], nstd_out))
    if np.size(outliers0_train)>0:
        X0_train_n = np.delete(X0_train_n,outliers0_train,axis=0)
        y0_train_n = np.delete(y0_train_n,outliers0_train)
    if np.size(outliers1_train)>0:
        X1_train_n = np.delete(X1_train_n,outliers1_train,axis=0)
        y1_train_n = np.delete(y1_train_n,outliers1_train)
    N_excl_tr = N_excl_tr + len(outliers0_train) + len(outliers1_train)


# SUBAMOSTRAGEM CLASSE 0
X0_train_n, X0_c, y0_train_n, y0_c = train_test_split(X0_train_n, y0_train_n, test_size=0.9, random_state=0)


# DADOS DE TREINO TOTAL
X_train_n = np.transpose(np.vstack((X0_train_n, X1_train_n)))
y_train_n = np.concatenate((y0_train_n, y1_train_n))
(L,N_train_n) = np.shape(X_train_n)
(L,N_val_n) = np.shape(X_val_n)

In [ ]:
# TRANSFORMA ESPACO
Ls_PCA = 55   # mse<5
w,mse,X_train_t,v = main.t4_pca(X_train_n, Ls_PCA)
X_val_t = np.dot(v.T,X_val_n)     # utiliza criterio dos dados de treino

ica = FastICA(n_components=Ls_PCA)
X_train_t = ica.fit_transform(np.transpose(X_train_t))  # estimated independent sources
X_val_t = ica.transform(np.transpose(X_val_t))
X_train_t = np.transpose(X_train_t)
X_val_t = np.transpose(X_val_t)

/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(


In [ ]:
# SELECAO ESCALAR COM RANDOM FOREST
arvores = RandomForestClassifier(n_estimators=200, min_samples_split=50, min_samples_leaf=20, random_state=0)
arvores.fit(np.transpose(X_train_t), y_train_n)
ordem = arvores.feature_importances_
ordem = np.argsort(-ordem)
Ls_SE = 40    # numero de caracteristicas a selecionar
X_train_s = np.zeros((Ls_SE,N_train_n))
X_val_s = np.zeros((Ls_SE,N_val_n))
for i in range(Ls_SE):
    X_train_s[i,:] = X_train_t[ordem[i],:]
    X_val_s[i,:] = X_val_t[ordem[i],:]    # utiliza criterio dos dados de treino


# SEPARA CLASSES
y1_train_t = y_train_n>0
X1_train_t = X_train_s[:,y1_train_t]
X0_train_t = np.delete(X_train_s,y1_train_t,axis=1)
y0_train_t = np.delete(y_train_n,y1_train_t)
y1_train_t = y_train_n[y1_train_t]
y1_val_t = y_val_n>0
X1_val_t = X_val_t[:,y1_val_t]
X0_val_t = np.delete(X_val_t,y1_val_t,axis=1)
y0_val_t = np.delete(y_val_n,y1_val_t)
y1_val_t = y_val_n[y1_val_t]


In [ ]:
l=0

#%% RANDOM FOREST
n_estimators = 1000
min_samples_split = 50
min_samples_leaf = 5
arvores = RandomForestClassifier(n_estimators=n_estimators, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, random_state=0)
arvores.fit(np.transpose(X_train_s), y_train_n)
y_train_pred = arvores.predict(np.transpose(X_train_s))
y_val_pred = arvores.predict(np.transpose(X_val_s))


# AVALIACAO
param={}
param['outlier'] = nstd_out
param['PCA-Ls'] = Ls_PCA
param['RT_Selec'] = Ls_SE
param['classificador'] = 'Random Forest'
param['n_estimators'] = n_estimators
param['min_samples_split'] = min_samples_split
param['min_samples_leaf'] = min_samples_leaf
train_aval={}
val_aval={}
train_aval['acc_train'] = accuracy_score(y_train_n, y_train_pred)
val_aval['acc_val'] = accuracy_score(y_val_n, y_val_pred)
train_aval['confMatrix_train'] = confusion_matrix(y_train_n, y_train_pred)
val_aval['confMatrix_val'] = confusion_matrix(y_val_n, y_val_pred)
c_target=1
train_aval['TP_train'] = train_aval['confMatrix_train'][c_target,c_target]
train_aval['FN_train'] = np.sum(train_aval['confMatrix_train'][c_target,:]) - train_aval['TP_train']
train_aval['TN_train'] = np.sum(train_aval['confMatrix_train'] * np.identity(2)) - train_aval['TP_train']
train_aval['FP_train'] = np.sum(train_aval['confMatrix_train']) - train_aval['TP_train'] - train_aval['FN_train'] - train_aval['TN_train']
train_aval['Sensibilidade_train'] = train_aval['TP_train']/(train_aval['TP_train']+train_aval['FN_train'])
train_aval['Especificidade_train'] = train_aval['TN_train']/(train_aval['TN_train']+train_aval['FP_train'])
train_aval['Precision_train'] = train_aval['TP_train']/(train_aval['TP_train']+train_aval['FP_train'])
train_aval['auc_train'] = roc_auc_score(y_train_n, y_train_pred)
val_aval['TP_val'] = val_aval['confMatrix_val'][c_target,c_target]
val_aval['FN_val'] = np.sum(val_aval['confMatrix_val'][c_target,:]) - val_aval['TP_val']
val_aval['TN_val'] = np.sum(val_aval['confMatrix_val'] * np.identity(2)) - val_aval['TP_val']
val_aval['FP_val'] = np.sum(val_aval['confMatrix_val']) - val_aval['TP_val'] - val_aval['FN_val'] - val_aval['TN_val']
val_aval['Sensibilidade_val'] = val_aval['TP_val']/(val_aval['TP_val']+val_aval['FN_val'])
val_aval['Especificidade_val'] = val_aval['TN_val']/(val_aval['TN_val']+val_aval['FP_val'])
val_aval['Precision_val'] = val_aval['TP_val']/(val_aval['TP_val']+val_aval['FP_val'])
val_aval['auc_val'] = roc_auc_score(y_val_n, y_val_pred)


# EXPORTA RESULTADOS
filename = '/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/aval/' + param['classificador']  + str(l) + '_train_aval.csv'
with open(filename, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in train_aval.items():
        writer.writerow([key, value])

filename = '/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/aval/' + param['classificador']  + str(l) + '_val_aval.csv'
with open(filename, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in val_aval.items():
        writer.writerow([key, value])

filename = '/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/aval/' + param['classificador']  + str(l) + '_parametros.csv'
with open(filename, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in param.items():
        writer.writerow([key, value])


# SVM RBF
C = 30
kernel = 'rbf'
tol = 0.01
gamma = 0.5
clf = SVC(C=C, kernel=kernel, tol=tol, gamma=gamma)
clf.fit(np.transpose(X_train_s), y_train_n)
y_train_pred = clf.predict(np.transpose(X_train_s))
y_val_pred = clf.predict(np.transpose(X_val_s))


# AVALIACAO
param={}
param['outlier'] = nstd_out
param['PCA-Ls'] = Ls_PCA
param['RT_Selec'] = Ls_SE
param['classificador'] = 'SVM - rbf'
param['C'] = C
param['kernel'] = kernel
param['tol'] = tol
param['gamma'] = gamma
train_aval={}
val_aval={}
train_aval['acc_train'] = accuracy_score(y_train_n, y_train_pred)
val_aval['acc_val'] = accuracy_score(y_val_n, y_val_pred)
train_aval['confMatrix_train'] = confusion_matrix(y_train_n, y_train_pred)
val_aval['confMatrix_val'] = confusion_matrix(y_val_n, y_val_pred)
c_target=1
train_aval['TP_train'] = train_aval['confMatrix_train'][c_target,c_target]
train_aval['FN_train'] = np.sum(train_aval['confMatrix_train'][c_target,:]) - train_aval['TP_train']
train_aval['TN_train'] = np.sum(train_aval['confMatrix_train'] * np.identity(2)) - train_aval['TP_train']
train_aval['FP_train'] = np.sum(train_aval['confMatrix_train']) - train_aval['TP_train'] - train_aval['FN_train'] - train_aval['TN_train']
train_aval['Sensibilidade_train'] = train_aval['TP_train']/(train_aval['TP_train']+train_aval['FN_train'])
train_aval['Especificidade_train'] = train_aval['TN_train']/(train_aval['TN_train']+train_aval['FP_train'])
train_aval['Precision_train'] = train_aval['TP_train']/(train_aval['TP_train']+train_aval['FP_train'])
train_aval['auc_train'] = roc_auc_score(y_train_n, y_train_pred)
val_aval['TP_val'] = val_aval['confMatrix_val'][c_target,c_target]
val_aval['FN_val'] = np.sum(val_aval['confMatrix_val'][c_target,:]) - val_aval['TP_val']
val_aval['TN_val'] = np.sum(val_aval['confMatrix_val'] * np.identity(2)) - val_aval['TP_val']
val_aval['FP_val'] = np.sum(val_aval['confMatrix_val']) - val_aval['TP_val'] - val_aval['FN_val'] - val_aval['TN_val']
val_aval['Sensibilidade_val'] = val_aval['TP_val']/(val_aval['TP_val']+val_aval['FN_val'])
val_aval['Especificidade_val'] = val_aval['TN_val']/(val_aval['TN_val']+val_aval['FP_val'])
val_aval['Precision_val'] = val_aval['TP_val']/(val_aval['TP_val']+val_aval['FP_val'])
val_aval['auc_val'] = roc_auc_score(y_val_n, y_val_pred)


# EXPORTA RESULTADOS
filename = '/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/aval/' + param['classificador']  + str(l) + '_train_aval.csv'
with open(filename, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in train_aval.items():
        writer.writerow([key, value])

filename = '/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/aval/' + param['classificador']  + str(l) + '_val_aval.csv'
with open(filename, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in val_aval.items():
        writer.writerow([key, value])

filename = '/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/aval/' + param['classificador']  + str(l) + '_parametros.csv'
with open(filename, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in param.items():
        writer.writerow([key, value])


# SVM POL
C = 20
kernel = 'poly'
degree = 4    #melhor no treino, pior no teste, sobreajuste
coef0 = 2
clf = SVC(C=C, kernel=kernel, degree=degree, coef0=coef0)
clf.fit(np.transpose(X_train_t), y_train_n)
y_train_pred = clf.predict(np.transpose(X_train_t))
y_val_pred = clf.predict(np.transpose(X_val_t))


# AVALIACAO
param={}
param['outlier'] = nstd_out
param['PCA-Ls'] = Ls_PCA
param['RT_Selec'] = Ls_SE
param['classificador'] = 'SVM - ploy'
param['C'] = C
param['kernel'] = kernel
param['degree'] = degree
param['coef0'] = coef0
train_aval={}
val_aval={}
train_aval['acc_train'] = accuracy_score(y_train_n, y_train_pred)
val_aval['acc_val'] = accuracy_score(y_val_n, y_val_pred)
train_aval['confMatrix_train'] = confusion_matrix(y_train_n, y_train_pred)
val_aval['confMatrix_val'] = confusion_matrix(y_val_n, y_val_pred)
c_target=1
train_aval['TP_train'] = train_aval['confMatrix_train'][c_target,c_target]
train_aval['FN_train'] = np.sum(train_aval['confMatrix_train'][c_target,:]) - train_aval['TP_train']
train_aval['TN_train'] = np.sum(train_aval['confMatrix_train'] * np.identity(2)) - train_aval['TP_train']
train_aval['FP_train'] = np.sum(train_aval['confMatrix_train']) - train_aval['TP_train'] - train_aval['FN_train'] - train_aval['TN_train']
train_aval['Sensibilidade_train'] = train_aval['TP_train']/(train_aval['TP_train']+train_aval['FN_train'])
train_aval['Especificidade_train'] = train_aval['TN_train']/(train_aval['TN_train']+train_aval['FP_train'])
train_aval['Precision_train'] = train_aval['TP_train']/(train_aval['TP_train']+train_aval['FP_train'])
train_aval['auc_train'] = roc_auc_score(y_train_n, y_train_pred)
val_aval['TP_val'] = val_aval['confMatrix_val'][c_target,c_target]
val_aval['FN_val'] = np.sum(val_aval['confMatrix_val'][c_target,:]) - val_aval['TP_val']
val_aval['TN_val'] = np.sum(val_aval['confMatrix_val'] * np.identity(2)) - val_aval['TP_val']
val_aval['FP_val'] = np.sum(val_aval['confMatrix_val']) - val_aval['TP_val'] - val_aval['FN_val'] - val_aval['TN_val']
val_aval['Sensibilidade_val'] = val_aval['TP_val']/(val_aval['TP_val']+val_aval['FN_val'])
val_aval['Especificidade_val'] = val_aval['TN_val']/(val_aval['TN_val']+val_aval['FP_val'])
val_aval['Precision_val'] = val_aval['TP_val']/(val_aval['TP_val']+val_aval['FP_val'])
val_aval['auc_val'] = roc_auc_score(y_val_n, y_val_pred)

# EXPORTA RESULTADOS
filename = '/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/aval/' + param['classificador']  + str(l) + '_train_aval.csv'
with open(filename, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in train_aval.items():
        writer.writerow([key, value])

filename = '/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/aval/' + param['classificador']  + str(l) + '_val_aval.csv'
with open(filename, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in val_aval.items():
        writer.writerow([key, value])

filename = '/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/aval/' + param['classificador']  + str(l) + '_parametros.csv'
with open(filename, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in param.items():
        writer.writerow([key, value])


# SVM SIGMOIDE
C = 30
kernel = 'sigmoid'
coef0 = -1
clf = SVC(C=C, kernel=kernel, coef0=coef0)
clf.fit(np.transpose(X_train_t), y_train_n)
y_train_pred = clf.predict(np.transpose(X_train_t))
y_val_pred = clf.predict(np.transpose(X_val_t))


# AVALIACAO
param={}
param['outlier'] = nstd_out
param['PCA-Ls'] = Ls_PCA
param['RT_Selec'] = Ls_SE
param['classificador'] = 'SVM - sig'
param['C'] = C
param['kernel'] = kernel
param['degree'] = degree
param['coef0'] = coef0
train_aval={}
val_aval={}
train_aval['acc_train'] = accuracy_score(y_train_n, y_train_pred)
val_aval['acc_val'] = accuracy_score(y_val_n, y_val_pred)
train_aval['confMatrix_train'] = confusion_matrix(y_train_n, y_train_pred)
val_aval['confMatrix_val'] = confusion_matrix(y_val_n, y_val_pred)
c_target=1
train_aval['TP_train'] = train_aval['confMatrix_train'][c_target,c_target]
train_aval['FN_train'] = np.sum(train_aval['confMatrix_train'][c_target,:]) - train_aval['TP_train']
train_aval['TN_train'] = np.sum(train_aval['confMatrix_train'] * np.identity(2)) - train_aval['TP_train']
train_aval['FP_train'] = np.sum(train_aval['confMatrix_train']) - train_aval['TP_train'] - train_aval['FN_train'] - train_aval['TN_train']
train_aval['Sensibilidade_train'] = train_aval['TP_train']/(train_aval['TP_train']+train_aval['FN_train'])
train_aval['Especificidade_train'] = train_aval['TN_train']/(train_aval['TN_train']+train_aval['FP_train'])
train_aval['Precision_train'] = train_aval['TP_train']/(train_aval['TP_train']+train_aval['FP_train'])
train_aval['auc_train'] = roc_auc_score(y_train_n, y_train_pred)
val_aval['TP_val'] = val_aval['confMatrix_val'][c_target,c_target]
val_aval['FN_val'] = np.sum(val_aval['confMatrix_val'][c_target,:]) - val_aval['TP_val']
val_aval['TN_val'] = np.sum(val_aval['confMatrix_val'] * np.identity(2)) - val_aval['TP_val']
val_aval['FP_val'] = np.sum(val_aval['confMatrix_val']) - val_aval['TP_val'] - val_aval['FN_val'] - val_aval['TN_val']
val_aval['Sensibilidade_val'] = val_aval['TP_val']/(val_aval['TP_val']+val_aval['FN_val'])
val_aval['Especificidade_val'] = val_aval['TN_val']/(val_aval['TN_val']+val_aval['FP_val'])
val_aval['Precision_val'] = val_aval['TP_val']/(val_aval['TP_val']+val_aval['FP_val'])
val_aval['auc_val'] = roc_auc_score(y_val_n, y_val_pred)


# EXPORTA RESULTADOS
filename = '/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/aval/' + param['classificador']  + str(l) + '_train_aval.csv'
with open(filename, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in train_aval.items():
        writer.writerow([key, value])

filename = '/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/aval/' + param['classificador']  + str(l) + '_val_aval.csv'
with open(filename, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in val_aval.items():
        writer.writerow([key, value])

filename = '/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/aval/' + param['classificador']  + str(l) + '_parametros.csv'
with open(filename, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in param.items():
        writer.writerow([key, value])


In [ ]:
l = 1
from keras.initializers import Zeros, Ones, glorot_uniform, he_normal
from keras.optimizers import Adam, SGD, RMSprop

opt = SGD(learning_rate=0.001)

# REDE NEURAL
D1 = Ls_SE
D2 = 120
D3 = 60
D4 = 1
optimizer = 'adam'
loss = 'binary_crossentropy'
batch_size = 50
epochs = 1000
model = Sequential()
#model.add(Dense(units=D1, kernel_initializer=he_normal(), activation='relu', input_dim=D1))
model.add(Dense(units=D1, activation='relu', input_dim=D1))
model.add(Dense(units=D2, activation='tanh', input_dim=D1))
model.add(Dense(units=D3, activation='tanh', input_dim=D2))
model.add(Dense(units=D4, activation='softmax', input_dim=D3))
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='accuracy', patience=250, restore_best_weights=True)   # define parada apos estagnacao de acuracia por 250 epocas, retoma melhores pesos
model.fit(np.transpose(X_train_s), y_train_n, validation_data=(np.transpose(X_val_s), y_val_n), batch_size=batch_size, epochs=epochs, callbacks=[early_stopping], verbose=1)
y_train_pred = model.predict(np.transpose(X_train_s))
y_val_pred = model.predict(np.transpose(X_val_s))
y_train_pred = (y_train_pred<0.5).astype(int)
y_val_pred = (y_val_pred<0.5).astype(int)


# AVALIACAO
param={}
param['outlier'] = nstd_out
param['PCA-Ls'] = Ls_PCA
param['RT_Selec'] = Ls_SE
param['classificador'] = 'Rede Neural'
param['D1'] = D1
param['D2'] = D2
param['D3'] = D3
param['D4'] = D4
param['epochs'] = epochs
param['batch_size'] = batch_size
param['optimezer'] = optimizer
param['loss'] = loss
train_aval={}
val_aval={}
train_aval['acc_train'] = accuracy_score(y_train_n, y_train_pred)
val_aval['acc_val'] = accuracy_score(y_val_n, y_val_pred)
train_aval['confMatrix_train'] = confusion_matrix(y_train_n, y_train_pred)
val_aval['confMatrix_val'] = confusion_matrix(y_val_n, y_val_pred)
c_target=1
train_aval['TP_train'] = train_aval['confMatrix_train'][c_target,c_target]
train_aval['FN_train'] = np.sum(train_aval['confMatrix_train'][c_target,:]) - train_aval['TP_train']
train_aval['TN_train'] = np.sum(train_aval['confMatrix_train'] * np.identity(2)) - train_aval['TP_train']
train_aval['FP_train'] = np.sum(train_aval['confMatrix_train']) - train_aval['TP_train'] - train_aval['FN_train'] - train_aval['TN_train']
train_aval['Sensibilidade_train'] = train_aval['TP_train']/(train_aval['TP_train']+train_aval['FN_train'])
train_aval['Especificidade_train'] = train_aval['TN_train']/(train_aval['TN_train']+train_aval['FP_train'])
train_aval['Precision_train'] = train_aval['TP_train']/(train_aval['TP_train']+train_aval['FP_train'])
train_aval['auc_train'] = roc_auc_score(y_train_n, y_train_pred)
val_aval['TP_val'] = val_aval['confMatrix_val'][c_target,c_target]
val_aval['FN_val'] = np.sum(val_aval['confMatrix_val'][c_target,:]) - val_aval['TP_val']
val_aval['TN_val'] = np.sum(val_aval['confMatrix_val'] * np.identity(2)) - val_aval['TP_val']
val_aval['FP_val'] = np.sum(val_aval['confMatrix_val']) - val_aval['TP_val'] - val_aval['FN_val'] - val_aval['TN_val']
val_aval['Sensibilidade_val'] = val_aval['TP_val']/(val_aval['TP_val']+val_aval['FN_val'])
val_aval['Especificidade_val'] = val_aval['TN_val']/(val_aval['TN_val']+val_aval['FP_val'])
val_aval['Precision_val'] = val_aval['TP_val']/(val_aval['TP_val']+val_aval['FP_val'])
val_aval['auc_val'] = roc_auc_score(y_val_n, y_val_pred)


# EXPORTA RESULTADOS
filename = '/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/aval/' + param['classificador']  + str(l) + '_train_aval.csv'
with open(filename, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in train_aval.items():
        writer.writerow([key, value])

filename = '/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/aval/' + param['classificador']  + str(l) + '_val_aval.csv'
with open(filename, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in val_aval.items():
        writer.writerow([key, value])

filename = '/content/drive/MyDrive/Pós/Aprendizado de Máquina/Projeto/aval/' + param['classificador']  + str(l) + '_parametros.csv'
with open(filename, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in param.items():
        writer.writerow([key, value])

